In [3]:
import cv2
import os
import numpy as np
from IPython.display import Image
import imutils
import tqdm
import re

# Question 1

## Fonctions utilisées

In [4]:
def charger(file):
    img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    return img

def montrer(img):
    cv2.imshow("image", img)
    k = cv2.waitKey(0)

def inserer(xBegin, yBegin, img, frag, angle=0):
    '''
    Insère frag dans img.

    (xBegin, yBegin) correspond à la position du point (0, 0) de frag dans img.
    '''

    if angle != 0 and angle != None:
        frag = tourner(frag, angle)
    img = ajouterCoucheAlpha(img)
    
    for x in range(0, frag.shape[0]):
        for y in range(0, frag.shape[1]):
            alpha_actuel = frag[x][y][3]
            if alpha_actuel:
                percentFrag = alpha_actuel/255
                img[xBegin + x, yBegin + y][0] = img[xBegin + x, yBegin + y, 0]*(1-percentFrag) + frag[x, y, 0]*percentFrag
                img[xBegin + x, yBegin + y][1] = img[xBegin + x, yBegin + y, 1]*(1-percentFrag) + frag[x, y, 1]*percentFrag
                img[xBegin + x, yBegin + y][2] = img[xBegin + x, yBegin + y, 2]*(1-percentFrag) + frag[x, y, 2]*percentFrag
            """else:
                img[xBegin + x, yBegin + y][0] = 0
                img[xBegin + x, yBegin + y][1] = 0
                img[xBegin + x, yBegin + y][2] = 0"""
    
    return img

def tourner(img, angle):
    h = img.shape[0]
    w = img.shape[1]
    imgRotated = imutils.rotate(img, angle)
    return imgRotated

def ajouterCoucheAlpha(img):
    '''
    Fait passer une image ayant 3 canaux de couleur mais pas de transparence,
        à une image avec 4 canaux : 3 pour la couleur et un pour la transparence.
        La transparence de l'image est initialisée à 0 (donc pas de transparence)
    '''
    
    if img.shape[2] == 3:
        img = np.pad(img, ((0, 0), (0, 0), (0, 1)), mode='constant', constant_values=0)
    return img

## Insertion des fragments sur la fresque originale pour être sûr qu'ils sont bien positionnés

In [5]:
#convertir fragments.txt en une hmap
hmap = {}
fichier_fragments_infos = open("tp1/fragments.txt", "r")
fragments_infos = fichier_fragments_infos.read()
fichier_fragments_infos.close()
fragments_infos = fragments_infos.split('\n')
for ligne in fragments_infos:
    try:
        ligne_coupee = ligne.split(' ')
        hmap[int(ligne_coupee[0])] = [
            int(ligne_coupee[1]),
            int(ligne_coupee[2]),
            float(ligne_coupee[3])
        ]
    except IndexError:
        pass
    except:
        1/0

angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
canvas = angelo
for nom_fichier in tqdm.tqdm(os.listdir('tp1/frag_eroded/')):
    fragment_index = re.findall(r'\d+', nom_fichier)[0]
    fragment_index = int(fragment_index)
    if fragment_index in list(hmap.keys()):
        fragment = charger(f'tp1/frag_eroded/{nom_fichier}')
        offset_x = -int(fragment.shape[0]/2)
        offset_y = -int(fragment.shape[1]/2)
        try:
            canvas = (inserer(hmap[fragment_index][1]-1 +  + offset_x, hmap[fragment_index][0]-1 + offset_y, canvas, fragment, angle=hmap[fragment_index][2]))
        except:
            pass
montrer(canvas)

100%|██████████| 328/328 [00:04<00:00, 70.24it/s] 


## Insertion des fragments sur un fond blanc pour simuler un recollage

In [6]:
#convertir fragments.txt en une hmap
hmap = {}
fichier_fragments_infos = open("tp1/fragments.txt", "r")
fragments_infos = fichier_fragments_infos.read()
fichier_fragments_infos.close()
fragments_infos = fragments_infos.split('\n')
for ligne in fragments_infos:
    try:
        ligne_coupee = ligne.split(' ')
        hmap[int(ligne_coupee[0])] = [
            int(ligne_coupee[1]),
            int(ligne_coupee[2]),
            float(ligne_coupee[3])
        ]
    except IndexError:
        pass
    except:
        1/0

fond_blanc = np.zeros(shape=(775, 1707, 4), dtype=np.uint8)
fond_blanc[:,:,:] = 255
canvas = fond_blanc
for nom_fichier in tqdm.tqdm(os.listdir('tp1/frag_eroded/')):
    fragment_index = re.findall(r'\d+', nom_fichier)[0]
    fragment_index = int(fragment_index)
    if fragment_index in list(hmap.keys()):
        fragment = charger(f'tp1/frag_eroded/{nom_fichier}')
        offset_x = -int(fragment.shape[0]/2)
        offset_y = -int(fragment.shape[1]/2)
        try:
            canvas = (inserer(hmap[fragment_index][1]-1 +  + offset_x, hmap[fragment_index][0]-1 + offset_y, canvas, fragment, angle=hmap[fragment_index][2]))
        except:
            pass
montrer(canvas)

100%|██████████| 328/328 [00:04<00:00, 75.04it/s] 


# Question 2

In [11]:
delta_x = 1     # unité : px
delta_y = 1     # unité : px
delta_alpha = 1 # unité : px


def bien_localise(x, y, alpha, x_barre, y_barre, alpha_barre):
    '''
    Vérifie si le fragment solution est bien localisé
    '''

    diff_x = np.abs(x - x_barre)
    diff_y = np.abs(y - y_barre)
    diff_alpha = np.abs(alpha - alpha_barre)

    if (diff_x <= delta_x) and (diff_y <= delta_y) and (diff_alpha <= delta_alpha):
        return True
    return False

def calculer_surface(frag):
    '''
    Calcule la vraie aire du fragment en regardant chaque pixel (très lent)
    '''
    res = 0     #surface en nombre de pixels
    for x in range(frag.shape[0]):
        for y in range(frag.shape[1]):
            if frag[x, y, 3] != 0:      #si le pixel n'est pas transperent
                res += 1                #il fait partie de la surface : on incrémente
    return res

def evaluer_solution(solution, fragments, vraieSurface = False):
    '''
    Parcourt le fichier solution fourni et calcule la qualité de la reconstitution en 
        comparant avec le fichier fragments.
    '''

    # on ouvre les fichiers fragments.txt et solutions.txt, 
    fichier_fragments_infos = open(fragments, "r")
    fragments_infos = fichier_fragments_infos.read()
    fichier_fragments_infos.close()


    fichier_solution_infos = open(solution, "r")
    solution_infos = fichier_solution_infos.read()
    fichier_solution_infos.close()


    fragments_infos = fragments_infos.split('\n')
    solution_infos = solution_infos.split('\n')

    #on crée une hmap pour réunir fragments_infos et solutions_infos, puis on appellera bien_localiser sur les valeurs de chaque clé de la hmap.
    hmap = {}

    for ligne in fragments_infos:
        try:
            ligne_coupee = ligne.split(' ')
            hmap[int(ligne_coupee[0])] = [
                int(ligne_coupee[1]),
                int(ligne_coupee[2]),
                float(ligne_coupee[3])
            ]
        except IndexError:
            pass
        except:
            1/0

    for ligne in solution_infos:
        try:
            ligne_coupee = ligne.split(' ')
            if len(ligne_coupee) == 4:
                curr_frag = charger(f"tp1/frag_eroded/frag_eroded_{ligne_coupee[0]}.png")
                surface = curr_frag.shape[0]*curr_frag.shape[1]
                if vraieSurface:
                    surface = calculer_surface(curr_frag)
                hmap[int(ligne_coupee[0])] += [
                    int(ligne_coupee[1]),
                    int(ligne_coupee[2]),
                    float(ligne_coupee[3]),
                    surface
                ]
        except IndexError:
            pass
        except:
            print(ligne_coupee)
            1/0


    res = 0
    surface_totale = 0
    for key in hmap.keys():
        #print(hmap[key])
        #print(hmap[key][0], hmap[key][3], hmap[key][1], hmap[key][4], hmap[key][2], hmap[key][5])
        surface_actuelle = hmap[key][6]
        surface_totale += surface_actuelle
        if bien_localise(hmap[key][0], hmap[key][1], hmap[key][2], hmap[key][3], hmap[key][4], hmap[key][5]):
            res += surface_actuelle
        else:
            res -= surface_actuelle

    res /= surface_totale
    
    return res


print(f'Qualité parfaite :                       {evaluer_solution("tp1/fragments.txt", "tp1/fragments.txt")}')
print(f'Qualité presque parfaite :               {evaluer_solution("tp1/solution.txt", "tp1/fragments.txt")}')
print(f'Qualité moins bonne :                    {evaluer_solution("tp1/solution_mauvaise_2.txt", "tp1/fragments.txt")}')
print(f'Qualité mauvaise :                       {evaluer_solution("tp1/solution_mauvaise_3.txt", "tp1/fragments.txt")}')
print(f'Qualité très mauvaise :                  {evaluer_solution("tp1/solution_mauvaise_4.txt", "tp1/fragments.txt")}')
print(f'Qualité très très mauvaise :             {evaluer_solution("tp1/solution_mauvaise_5.txt", "tp1/fragments.txt")}')
print(f'Qualité très très très mauvaise :        {evaluer_solution("tp1/solution_mauvaise_6.txt", "tp1/fragments.txt")}')
print(f'Qualité d\'une solution 100% aléatoire : {evaluer_solution("tp1/solution_mauvaise_1.txt", "tp1/fragments.txt")}')

print(f'Qualité parfaite avec vraieSurface :     {evaluer_solution("tp1/fragments.txt", "tp1/fragments.txt", vraieSurface=True)}')
print(f'Qualité moins bonne avec vraieSurface :  {evaluer_solution("tp1/solution_mauvaise_2.txt", "tp1/fragments.txt", vraieSurface=True)}')

Qualité parfaite :                       1.0
Qualité presque parfaite :               0.9802758354639558
Qualité moins bonne :                    0.8419487885481706
Qualité mauvaise :                       0.644866053693024
Qualité très mauvaise :                  0.3188047040719881
Qualité très très mauvaise :             -0.1679766974063077
Qualité très très très mauvaise :        -0.7602479217903488
Qualité d'une solution 100% aléatoire : -1.0
Qualité parfaite avec vraieSurface :     1.0
Qualité moins bonne avec vraieSurface :  0.8419487885481706
